In [2]:
import pandas as pd
import numpy as np
import seaborn as sns


In [3]:
df_preprocessed = pd.read_csv('df_preprocessed.csv')

In [4]:
X=df_preprocessed.drop('Life expectancy',axis=1)
y=df_preprocessed['Life expectancy']

In [5]:
#feature selection
from sklearn.feature_selection import SelectKBest, f_regression

# select the top 22 features based on f_regression
selector = SelectKBest(f_regression, k=22)
X_selected = selector.fit_transform(X, y)

# get the indices of the selected features
selected_indices = selector.get_support(indices=True)

# get the names of the selected features
selected_features = X.columns[selected_indices]


In [6]:
df_selected=X[selected_features]

In [7]:
df_selected.columns

Index(['Country_Angola', 'Country_Central African Republic', 'Country_Chad',
       'Country_Côte d'Ivoire', 'Country_Lesotho', 'Country_Malawi',
       'Country_Nigeria', 'Country_Zimbabwe', 'Status_Developed',
       'Status_Developing', 'Adult Mortality', 'infant deaths', 'Alcohol',
       'percentage expenditure', 'BMI', 'under-five deaths', 'Polio',
       'Diphtheria', 'HIV/AIDS', 'thinness  1-19 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')

In [8]:
#test train split
from sklearn.model_selection import train_test_split

# Define the features and target variable
X = df_selected
y = df_preprocessed['Life expectancy']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shape of the training and testing sets
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)


Training data shape: (2342, 22)
Testing data shape: (586, 22)


In [22]:
import xgboost as xgb


xgb_reg = xgb.XGBRegressor()

# Train the model on the training set
xgb_reg.fit(X_train, y_train)

# Evaluate the model on the testing set
score = xgb_reg.score(X_test, y_test)

print("Model score: ", score)


Model score:  0.776516149688448


In [26]:

from sklearn.model_selection import GridSearchCV

xgb_reg = xgb.XGBRegressor()

# Define the parameter grid to search
param_grid = {
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9]
}

# Create a GridSearchCV object and fit it to the data
grid_search = GridSearchCV(xgb_reg, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Train the model on the training set with the best parameters
best_xgb_reg = xgb.XGBRegressor(**grid_search.best_params_)
best_xgb_reg.fit(X_train, y_train)

# Evaluate the model on the testing set
score = best_xgb_reg.score(X_test, y_test)
print("Model score: ", score)


Best parameters:  {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 500, 'subsample': 0.9}
Best score:  0.791330765175187
Model score:  0.7894938867899615


In [27]:
from catboost import CatBoostRegressor

# Create a CatBoostRegressor model
cat_reg = CatBoostRegressor()

# Train the model on the training set
cat_reg.fit(X_train, y_train)

# Evaluate the model on the testing set
score = cat_reg.score(X_test, y_test)

print("Model score: ", score)


Learning rate set to 0.046835
0:	learn: 9.2468684	total: 229ms	remaining: 3m 48s
1:	learn: 9.0613722	total: 233ms	remaining: 1m 56s
2:	learn: 8.8742317	total: 237ms	remaining: 1m 18s
3:	learn: 8.7042558	total: 241ms	remaining: 60s
4:	learn: 8.5388311	total: 244ms	remaining: 48.6s
5:	learn: 8.3762624	total: 248ms	remaining: 41.1s
6:	learn: 8.2272566	total: 252ms	remaining: 35.8s
7:	learn: 8.0862907	total: 256ms	remaining: 31.7s
8:	learn: 7.9584320	total: 259ms	remaining: 28.6s
9:	learn: 7.8336307	total: 263ms	remaining: 26s
10:	learn: 7.7172801	total: 267ms	remaining: 24s
11:	learn: 7.6164092	total: 272ms	remaining: 22.4s
12:	learn: 7.5129425	total: 276ms	remaining: 21s
13:	learn: 7.4155933	total: 280ms	remaining: 19.7s
14:	learn: 7.3246362	total: 285ms	remaining: 18.7s
15:	learn: 7.2438351	total: 289ms	remaining: 17.8s
16:	learn: 7.1609843	total: 292ms	remaining: 16.9s
17:	learn: 7.0894462	total: 296ms	remaining: 16.2s
18:	learn: 7.0099620	total: 300ms	remaining: 15.5s
19:	learn: 6.949

161:	learn: 4.6533382	total: 890ms	remaining: 4.6s
162:	learn: 4.6493034	total: 895ms	remaining: 4.59s
163:	learn: 4.6419903	total: 899ms	remaining: 4.58s
164:	learn: 4.6316197	total: 904ms	remaining: 4.57s
165:	learn: 4.6282283	total: 908ms	remaining: 4.56s
166:	learn: 4.6220127	total: 911ms	remaining: 4.54s
167:	learn: 4.6146849	total: 915ms	remaining: 4.53s
168:	learn: 4.6052972	total: 918ms	remaining: 4.52s
169:	learn: 4.5983014	total: 922ms	remaining: 4.5s
170:	learn: 4.5879544	total: 926ms	remaining: 4.49s
171:	learn: 4.5808719	total: 929ms	remaining: 4.47s
172:	learn: 4.5720842	total: 933ms	remaining: 4.46s
173:	learn: 4.5665415	total: 936ms	remaining: 4.45s
174:	learn: 4.5632348	total: 940ms	remaining: 4.43s
175:	learn: 4.5547525	total: 943ms	remaining: 4.41s
176:	learn: 4.5500258	total: 946ms	remaining: 4.4s
177:	learn: 4.5462472	total: 950ms	remaining: 4.39s
178:	learn: 4.5345451	total: 954ms	remaining: 4.38s
179:	learn: 4.5301087	total: 957ms	remaining: 4.36s
180:	learn: 4.5

338:	learn: 3.6921452	total: 1.55s	remaining: 3.02s
339:	learn: 3.6857930	total: 1.55s	remaining: 3.02s
340:	learn: 3.6792168	total: 1.56s	remaining: 3.01s
341:	learn: 3.6741456	total: 1.56s	remaining: 3.01s
342:	learn: 3.6690151	total: 1.57s	remaining: 3.01s
343:	learn: 3.6655006	total: 1.57s	remaining: 3s
344:	learn: 3.6611893	total: 1.58s	remaining: 2.99s
345:	learn: 3.6595240	total: 1.58s	remaining: 2.98s
346:	learn: 3.6535919	total: 1.58s	remaining: 2.98s
347:	learn: 3.6475683	total: 1.59s	remaining: 2.97s
348:	learn: 3.6435841	total: 1.59s	remaining: 2.97s
349:	learn: 3.6405996	total: 1.59s	remaining: 2.96s
350:	learn: 3.6373042	total: 1.6s	remaining: 2.95s
351:	learn: 3.6346575	total: 1.6s	remaining: 2.95s
352:	learn: 3.6333028	total: 1.6s	remaining: 2.94s
353:	learn: 3.6301818	total: 1.61s	remaining: 2.93s
354:	learn: 3.6246312	total: 1.61s	remaining: 2.93s
355:	learn: 3.6204936	total: 1.61s	remaining: 2.92s
356:	learn: 3.6140825	total: 1.62s	remaining: 2.91s
357:	learn: 3.6106

517:	learn: 3.0475423	total: 2.21s	remaining: 2.06s
518:	learn: 3.0435631	total: 2.22s	remaining: 2.05s
519:	learn: 3.0396872	total: 2.22s	remaining: 2.05s
520:	learn: 3.0369878	total: 2.23s	remaining: 2.05s
521:	learn: 3.0323487	total: 2.23s	remaining: 2.04s
522:	learn: 3.0291463	total: 2.23s	remaining: 2.04s
523:	learn: 3.0256670	total: 2.24s	remaining: 2.03s
524:	learn: 3.0221565	total: 2.24s	remaining: 2.03s
525:	learn: 3.0194617	total: 2.25s	remaining: 2.02s
526:	learn: 3.0171863	total: 2.25s	remaining: 2.02s
527:	learn: 3.0140135	total: 2.25s	remaining: 2.01s
528:	learn: 3.0108652	total: 2.26s	remaining: 2.01s
529:	learn: 3.0058077	total: 2.26s	remaining: 2s
530:	learn: 3.0026832	total: 2.26s	remaining: 2s
531:	learn: 2.9995976	total: 2.27s	remaining: 1.99s
532:	learn: 2.9949434	total: 2.27s	remaining: 1.99s
533:	learn: 2.9921303	total: 2.27s	remaining: 1.98s
534:	learn: 2.9888406	total: 2.28s	remaining: 1.98s
535:	learn: 2.9855649	total: 2.28s	remaining: 1.97s
536:	learn: 2.9831

698:	learn: 2.5909719	total: 2.87s	remaining: 1.24s
699:	learn: 2.5892427	total: 2.88s	remaining: 1.23s
700:	learn: 2.5876877	total: 2.88s	remaining: 1.23s
701:	learn: 2.5851388	total: 2.89s	remaining: 1.23s
702:	learn: 2.5836578	total: 2.89s	remaining: 1.22s
703:	learn: 2.5803107	total: 2.9s	remaining: 1.22s
704:	learn: 2.5784410	total: 2.9s	remaining: 1.21s
705:	learn: 2.5757662	total: 2.9s	remaining: 1.21s
706:	learn: 2.5736838	total: 2.91s	remaining: 1.2s
707:	learn: 2.5720083	total: 2.91s	remaining: 1.2s
708:	learn: 2.5685468	total: 2.91s	remaining: 1.2s
709:	learn: 2.5650429	total: 2.92s	remaining: 1.19s
710:	learn: 2.5627622	total: 2.92s	remaining: 1.19s
711:	learn: 2.5607274	total: 2.92s	remaining: 1.18s
712:	learn: 2.5589746	total: 2.93s	remaining: 1.18s
713:	learn: 2.5562791	total: 2.93s	remaining: 1.17s
714:	learn: 2.5544374	total: 2.94s	remaining: 1.17s
715:	learn: 2.5541806	total: 2.94s	remaining: 1.17s
716:	learn: 2.5515491	total: 2.94s	remaining: 1.16s
717:	learn: 2.5499

871:	learn: 2.2633880	total: 3.54s	remaining: 520ms
872:	learn: 2.2591353	total: 3.54s	remaining: 516ms
873:	learn: 2.2575064	total: 3.55s	remaining: 512ms
874:	learn: 2.2559574	total: 3.56s	remaining: 508ms
875:	learn: 2.2544455	total: 3.56s	remaining: 504ms
876:	learn: 2.2539006	total: 3.56s	remaining: 500ms
877:	learn: 2.2535250	total: 3.57s	remaining: 496ms
878:	learn: 2.2527746	total: 3.57s	remaining: 492ms
879:	learn: 2.2512212	total: 3.57s	remaining: 487ms
880:	learn: 2.2496115	total: 3.58s	remaining: 483ms
881:	learn: 2.2484508	total: 3.58s	remaining: 479ms
882:	learn: 2.2471253	total: 3.58s	remaining: 475ms
883:	learn: 2.2450895	total: 3.59s	remaining: 471ms
884:	learn: 2.2436916	total: 3.59s	remaining: 467ms
885:	learn: 2.2416701	total: 3.6s	remaining: 463ms
886:	learn: 2.2400830	total: 3.6s	remaining: 458ms
887:	learn: 2.2381057	total: 3.6s	remaining: 454ms
888:	learn: 2.2375249	total: 3.6s	remaining: 450ms
889:	learn: 2.2354525	total: 3.61s	remaining: 446ms
890:	learn: 2.23

In [9]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

# Create a CatBoostRegressor object
cat_reg = CatBoostRegressor(verbose=False)

# Define the hyperparameter grid to search over
param_grid = {'learning_rate': [0.01, 0.05, 0.1],
              'depth': [3, 5, 7],
              'l2_leaf_reg': [1, 3, 5],
              'iterations': [100, 500, 1000]}

# Create a GridSearchCV object to find the best hyperparameters
grid_search = GridSearchCV(cat_reg, param_grid, cv=5, n_jobs=-1)

# Train the model on the training set
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Evaluate the model on the testing set using the best hyperparameters
best_cat_reg = grid_search.best_estimator_
score = best_cat_reg.score(X_test, y_test)
print("Model score: ", score)


Best parameters:  {'depth': 7, 'iterations': 1000, 'l2_leaf_reg': 5, 'learning_rate': 0.1}
Best score:  0.7914322788788799
Model score:  0.8138055064158853


In [11]:
import pickle

# open a file in write-binary mode to save the model
with open('predictor_model.pkl', 'wb') as f:
    
    pickle.dump(best_cat_reg, f)

# close the file
f.close()
